#### Cell 1 — Import Libraries & Load Dataset

In [3]:
import pandas as pd
import numpy as np


In [4]:
# Load dataset
df = pd.read_csv("dirty_financial_transactions.csv")

# View basic information
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Transaction_ID      94982 non-null   object 
 1   Transaction_Date    95120 non-null   object 
 2   Customer_ID         95122 non-null   object 
 3   Product_Name        100000 non-null  object 
 4   Quantity            94981 non-null   float64
 5   Price               66503 non-null   object 
 6   Payment_Method      100000 non-null  object 
 7   Transaction_Status  83321 non-null   object 
dtypes: float64(1), object(7)
memory usage: 6.1+ MB


In [5]:
df.shape

(100000, 8)

In [6]:
# Preview first 5 rows
df.head()


,Transaction_ID,Transaction_Date,Customer_ID,Product_Name,Quantity,Price,Payment_Method,Transaction_Status
0,T0001,2024-08-02,C2205,Headphones,-5.0,$420.21,pay pal,NaN
1,T0002,2020-02-10,C3156,Coffee,469.0,-445.34202525395585,creditcard,Pending
2,T0003,2025-02-30,C2919,Tablet,-4.0,810.9930123946459,credit card,completed
3,T0004,2020-08-17,C3009,Tab,-7.0,868.6083413217348,PayPal,Pending
4,T0005,2025-02-30,C3488,Coffee Machine,-10.0,-763.1224490039416,PayPal,completed


#### Cell 2 — Check Missing (Blank) Values

In [7]:
# Check missing values in each column
df.isnull().sum()


Transaction_ID         5018
Transaction_Date       4880
Customer_ID            4878
Product_Name              0
Quantity               5019
Price                 33497
Payment_Method            0
Transaction_Status    16679
dtype: int64

#### Cell 3 — Remove Rows with Critical Missing Data

In [8]:
df = df.dropna(subset=[
    "Transaction_ID",
    "Transaction_Date",
    "Customer_ID"
])


#### Transaction_Date column

##### Convert to DateTime

In [9]:
# Convert Transaction_Date to datetime
df["Transaction_Date"] = pd.to_datetime(
    df["Transaction_Date"],
    errors="coerce"
)

##### Remove Invalid Dates

In [10]:
df = df.dropna(subset=["Transaction_Date"])

##### Create Year, Month, Day Columns

In [11]:
df["Year"] = df["Transaction_Date"].dt.year
df["Month"] = df["Transaction_Date"].dt.month
df["Day"] = df["Transaction_Date"].dt.day

##### Create Month Name

In [12]:
df["Month_Name"] = df["Transaction_Date"].dt.month_name()

##### Create Year-Month Column

In [13]:
df["Year_Month"] = df["Transaction_Date"].dt.to_period("M").astype(str)


##### Final Validation

In [14]:
df[[
    "Transaction_Date",
    "Year",
    "Month",
    "Month_Name",
    "Year_Month"
]].head()


,Transaction_Date,Year,Month,Month_Name,Year_Month
0,2024-08-02,2024,8,August,2024-08
1,2020-02-10,2020,2,February,2020-02
3,2020-08-17,2020,8,August,2020-08
5,2021-10-26,2021,10,October,2021-10
12,2020-03-18,2020,3,March,2020-03


In [15]:
df["Transaction_Date"].dtype


dtype('<M8[ns]')

### Price Column

##### Clean and Convert Price Column

In [16]:
# Remove currency symbols and spaces
df["Price"] = (
    df["Price"]
    .astype(str)
    .str.replace("$", "", regex=False)
    .str.strip()
)

# Convert to numeric (invalid values become NaN)
df["Price"] = pd.to_numeric(df["Price"], errors="coerce")


#### Remove Rows with Missing Price

In [17]:
df = df.dropna(subset=["Price"])

#### Create Transaction Type Column

In [18]:
df["Transaction_Type"] = np.where(
    df["Price"] < 0,
    "Refund/Reversal",
    "Sale"
)


#### Create Absolute Price Column

In [19]:
df["Abs_Price"] = df["Price"].abs()

##### Check Refund Values (Negative Price)

In [20]:
df[df["Price"] < 0][["Transaction_ID", "Product_Name", "Price"]].head()

,Transaction_ID,Product_Name,Price
1,T0002,Coffee,-445.342025
16,T0017,Smartphone,-94.557265
51,T0052,Headphones,-756.334077
68,T0069,Headphones,-750.983852
80,T0081,Smartphone,-845.030640


##### Create Refund_Amount Column

In [21]:
df["Refund_Amount"] = df["Price"].apply(
    lambda x: abs(x) if x < 0 else 0
)


##### Create Sales_Amount Column

In [22]:
df["Sales_Amount"] = df["Price"].apply(
    lambda x: x if x > 0 else 0
)

#### Check Result

In [23]:
df[["Price", "Sales_Amount", "Refund_Amount"]].head(10)

,Price,Sales_Amount,Refund_Amount
0,420.210000,420.210000,0.000000
1,-445.342025,0.000000,445.342025
3,868.608341,868.608341,0.000000
13,905.514730,905.514730,0.000000
16,-94.557265,0.000000,94.557265
24,276.936320,276.936320,0.000000
30,787.994076,787.994076,0.000000
34,970.280000,970.280000,0.000000
51,-756.334077,0.000000,756.334077
52,154.185054,154.185054,0.000000


#### Remove Null Quantity Rows

In [24]:
df = df.dropna(subset=["Quantity"])

#### Convert Quantity to Numeric

In [25]:
df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")

#### Convert Negative Quantity to Positive

In [26]:
df["Quantity"] = df["Quantity"].abs()

In [27]:
df["Quantity"].describe()

count    18139.000000
mean       189.260764
std        299.144526
min          1.000000
25%          4.000000
50%          8.000000
75%        332.000000
max       1000.000000
Name: Quantity, dtype: float64

### Transaction_Status

#### Inspect Current Status Values

In [28]:
df["Transaction_Status"].unique()


array([nan, 'Pending', 'Failed', 'completed', 'Completed', 'complete'],
      dtype=object)

#### Basic Text Cleaning

In [29]:
df["Transaction_Status"] = (
    df["Transaction_Status"]
    .astype(str)
    .str.lower()
    .str.strip()
)

#### Standardise Status Labels

In [30]:
df["Transaction_Status"] = df["Transaction_Status"].replace({
    "complete": "completed",
    "complted": "completed",
    "done": "completed",
    "pendng": "pending"
})

#### Calculate Mode of Transaction_Status

In [31]:
most_frequent_status = df["Transaction_Status"].mode()[0]
most_frequent_status

'completed'

#### Fill Missing Status Values

In [32]:
df["Transaction_Status"] = df["Transaction_Status"].replace("nan", np.nan)
df["Transaction_Status"] = df["Transaction_Status"].fillna(most_frequent_status)


#### Validate the Result

In [33]:
df["Transaction_Status"].isnull().sum()

np.int64(0)

In [34]:
df["Transaction_Status"].value_counts()

Transaction_Status
completed    12088
pending       3056
failed        2995
Name: count, dtype: int64

### payment method

##### Inspect Current Values

In [35]:
df["Payment_Method"].unique()

array(['pay pal', 'creditcard', 'PayPal', 'credit card', 'Credit Card',
       'PayPal ', 'Cash'], dtype=object)

##### Clean and Format Payment_Method

In [36]:
df["Payment_Method"] = (
    df["Payment_Method"]
    .astype(str)
    .str.lower()
    .str.strip()
    .str.capitalize()
)


### Validate the Result

In [37]:
df["Payment_Method"].unique()

array(['Pay pal', 'Creditcard', 'Paypal', 'Credit card', 'Cash'],
      dtype=object)

In [38]:
df["Payment_Method"].value_counts()

Payment_Method
Credit card    5210
Paypal         5187
Creditcard     2656
Pay pal        2571
Cash           2515
Name: count, dtype: int64

#### Product Name Column

#### Check Unique Product Names

In [39]:
sorted(df["Product_Name"].unique())

['C',
 'Co',
 'Cof',
 'Coff',
 'Coffe',
 'Coffee',
 'Coffee ',
 'Coffee M',
 'Coffee Ma',
 'Coffee Mac',
 'Coffee Mach',
 'Coffee Machi',
 'Coffee Machin',
 'Coffee Machine',
 'H',
 'He',
 'Hea',
 'Head',
 'Headp',
 'Headph',
 'Headpho',
 'Headphon',
 'Headphone',
 'Headphones',
 'L',
 'La',
 'Lap',
 'Lapt',
 'Lapto',
 'Laptop',
 'S',
 'Sm',
 'Sma',
 'Smar',
 'Smart',
 'Smartp',
 'Smartph',
 'Smartpho',
 'Smartphon',
 'Smartphone',
 'T',
 'Ta',
 'Tab',
 'Tabl',
 'Table',
 'Tablet']

##### Clean Product_Name Text

In [40]:
df["Product_Name"] = (
    df["Product_Name"]
    .astype(str)
    .str.strip()
    .str.lower()
)

##### Create Product Cleaning Function

In [41]:
def clean_product_name(name):
    if name.startswith("c"):
        return "Coffee Machine"
    elif name.startswith("l"):
        return "Laptop"
    elif name.startswith("s"):
        return "Smartphone"
    elif name.startswith("t"):
        return "Tablet"
    elif name.startswith("h"):
        return "Headphones"
    else:
        return name.title()


##### Apply Mapping to Product_Name

In [42]:
df["Product_Name"] = df["Product_Name"].apply(clean_product_name)

##### Check Cleaned Product Names

In [43]:
df["Product_Name"].value_counts()

Product_Name
Tablet            3722
Coffee Machine    3667
Laptop            3636
Smartphone        3571
Headphones        3543
Name: count, dtype: int64

In [44]:
df.to_csv("cleaned_financial_transactions.csv", index=False)